## Keypoint Detection Solution
---

In [2]:
# load packages
import cv2
import numpy as np
import os
import torch
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import re
%matplotlib inline
print("Done!")

Done!


In [3]:
# Initiate SIFT detector
sift = cv2.xfeatures2d.SIFT_create(30)
surf = cv2.xfeatures2d.SURF_create(300)
print("Done!")

Done!


In [4]:
def getPatches(kps, img, size=32, num=500):
    res = torch.zeros(num, 1, size, size)
    if type(img) is np.ndarray:
        img = torch.from_numpy(img)
    h, w = img.shape      # note: for image, the x direction is the verticle, y-direction is the horizontal...
    for i in range(num):
        cx, cy = kps[i]
        cx, cy = int(cx), int(cy)
        dd = int(size/2)
        xmin, xmax = max(0, cx - dd), min(w, cx + dd ) - 1
        ymin, ymax = max(0, cy - dd), min(h, cy + dd ) - 1 
        
        xmin_res, xmax_res = dd - min(dd,cx), dd + min(dd, w - cx)-1
        ymin_res, ymax_res = dd - min(dd,cy), dd + min(dd, h - cy)-1

        res[i, 0, ymin_res: ymax_res, xmin_res: xmax_res] = img[ymin: ymax, xmin: xmax]
    return res
print("Done!")

Done!


In [25]:
img_dir2 = "../image_retrieval/images"
img_dir1 = "../image_retrieval/query"
#img_dir = "../image_retrieval/all"

kps_num = 30
patch_size = 32
#number_of_images = 140
#number_of_images = 35
number_of_images = 175
res = torch.zeros(number_of_images, kps_num, 2)
patches = torch.zeros(number_of_images, kps_num, 1, patch_size, patch_size)
if os.path.exists(img_dir):
    if os.listdir(img_dir) is []:
        print("No images!")
        exit(0)
    num_img = len(os.listdir(img_dir))
    idx = 0
    sorted_listdir1 = sorted(os.listdir(img_dir1), key=lambda e: [int(s) for s in re.split('[q.]', e) if s.isdigit()])
    sorted_listdir2 = sorted(os.listdir(img_dir2), key=lambda e: [int(s) for s in re.split('[q.]', e) if s.isdigit()])
    for img in sorted_listdir1:
        print(img)
        if not img.endswith("JPG"):
            continue
        image_dir = os.path.join(img_dir, img)
        image = cv2.imread(image_dir)
        img= cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        
        ## find the keypoints and descriptors with SIFT
        if (image_dir != "../image_retrieval/all\q17.JPG" and image_dir != "../image_retrieval/query\q17.JPG"):
            kps, des = sift.detectAndCompute(img, None)
            keypoints_img = [kps[a].pt for a in range(kps_num)] 
            res[idx] = torch.FloatTensor(keypoints_img)
        else:
            kps, des = surf.detectAndCompute(img, None)
            keypoints_img = [kps[a].pt for a in range(kps_num)] 
            res[idx] = torch.FloatTensor(keypoints_img)
        ## extract patches
        patches[idx] = getPatches(keypoints_img, img, size=patch_size, num=kps_num)
        idx += 1
    for img in sorted_listdir2:
        print(img)
        if not img.endswith("JPG"):
            continue
        image_dir = os.path.join(img_dir, img)
        image = cv2.imread(image_dir)
        img= cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        
        ## find the keypoints and descriptors with SIFT
        if (image_dir != "../image_retrieval/all\q17.JPG" and image_dir != "../image_retrieval/query\q17.JPG"):
            kps, des = sift.detectAndCompute(img, None)
            keypoints_img = [kps[a].pt for a in range(kps_num)] 
            res[idx] = torch.FloatTensor(keypoints_img)
        else:
            kps, des = surf.detectAndCompute(img, None)
            keypoints_img = [kps[a].pt for a in range(kps_num)] 
            res[idx] = torch.FloatTensor(keypoints_img)
        ## extract patches
        patches[idx] = getPatches(keypoints_img, img, size=patch_size, num=kps_num)
        idx += 1
else:
    print("image folder not exists!")
    exit(0)
print("Done!")

q1.JPG
q2.JPG
q3.JPG
q4.JPG
q5.JPG
q6.JPG
q7.JPG
q8.JPG
q9.JPG
q10.JPG
q11.JPG
q12.JPG
q13.JPG
q14.JPG
q15.JPG
q16.JPG
q17.JPG
q18.JPG
q19.JPG
q20.JPG
q21.JPG
q22.JPG
q23.JPG
q24.JPG
q25.JPG
q26.JPG
q27.JPG
q28.JPG
q29.JPG
q30.JPG
q31.JPG
q32.JPG
q33.JPG
q34.JPG
q35.JPG
1.JPG
2.JPG
3.JPG
4.JPG
5.JPG
6.JPG
7.JPG
8.JPG
9.JPG
10.JPG
11.JPG
12.JPG
13.JPG
14.JPG
15.JPG
16.JPG
17.JPG
18.JPG
19.JPG
20.JPG
21.JPG
22.JPG
23.JPG
24.JPG
25.JPG
26.JPG
27.JPG
28.JPG
29.JPG
30.JPG
31.JPG
32.JPG
33.JPG
34.JPG
35.JPG
36.JPG
37.JPG
38.JPG
39.JPG
40.JPG
41.JPG
42.JPG
43.JPG
44.JPG
45.JPG
46.JPG
47.JPG
48.JPG
49.JPG
50.JPG
51.JPG
52.JPG
53.JPG
54.JPG
55.JPG
56.JPG
57.JPG
58.JPG
59.JPG
60.JPG
61.JPG
62.JPG
63.JPG
64.JPG
65.JPG
66.JPG
67.JPG
68.JPG
69.JPG
70.JPG
71.JPG
72.JPG
73.JPG
74.JPG
75.JPG
76.JPG
77.JPG
78.JPG
79.JPG
80.JPG
81.JPG
82.JPG
83.JPG
84.JPG
85.JPG
86.JPG
87.JPG
88.JPG
89.JPG
90.JPG
91.JPG
92.JPG
93.JPG
94.JPG
95.JPG
96.JPG
97.JPG
98.JPG
99.JPG
100.JPG
101.JPG
102.JPG
103.JPG
104.JPG
105.J

In [27]:
print(res.shape)
print(patches.shape)
print(res)
print(patches)
print("Done!")

torch.Size([175, 30, 2])
torch.Size([175, 30, 1, 32, 32])
tensor([[[131.0552,  93.5981],
         [140.6713,  99.0963],
         [140.6713,  99.0963],
         ...,
         [133.2535, 128.4837],
         [142.1921, 140.7693],
         [266.0648, 121.2904]],

        [[139.6310,  99.4554],
         [139.6310,  99.4554],
         [188.3795,  96.2190],
         ...,
         [181.5607, 103.8010],
         [ 78.3812, 198.3145],
         [100.9016, 109.3896]],

        [[233.5516, 166.9911],
         [238.2965, 147.6480],
         [ 51.5890, 129.6989],
         ...,
         [141.4853,  49.4803],
         [212.8207, 120.8577],
         [212.8207, 120.8577]],

        ...,

        [[214.1256,  60.2871],
         [190.6586, 115.3140],
         [190.6586, 115.3140],
         ...,
         [211.8242, 111.1548],
         [211.8242, 111.1548],
         [187.8636,  81.4924]],

        [[187.5141,  79.7814],
         [213.9660,  38.5946],
         [162.1113,  90.5041],
         ...,
         [182

In [29]:
## save tensors
#output_dir_kps = "keypoints-i.pt"
#output_dir_patches = "patches-i.pt"
#output_dir_kps = "keypoints-q.pt"
#output_dir_patches = "patches-q.pt"
output_dir_kps = "keypoints-a.pt"
output_dir_patches = "patches-a.pt"
torch.save(res, output_dir_kps)
torch.save(patches, output_dir_patches)
print("Done!")

Done!
